<a href="https://colab.research.google.com/github/Jason1Borne/ML_practice/blob/master/GAL_kaggle_to_subject_final_edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import kaggle.json to download the dataset

In [0]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"szutang","key":"a789eb8084078329e99d1ba0960540c5"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c grasp-and-lift-eeg-detection

 97% 5.00M/5.16M [00:00<00:00, 34.0MB/s]
100% 5.16M/5.16M [00:00<00:00, 33.1MB/s]
 90% 137M/153M [00:00<00:00, 127MB/s]
100% 153M/153M [00:01<00:00, 150MB/s]
 98% 894M/915M [00:09<00:00, 92.6MB/s]
100% 915M/915M [00:09<00:00, 102MB/s] 


In [0]:
!unzip train.zip

Archive:  train.zip
   creating: train/
  inflating: train/subj10_series1_data.csv  
  inflating: train/subj10_series1_events.csv  
  inflating: train/subj10_series2_data.csv  
  inflating: train/subj10_series2_events.csv  
  inflating: train/subj10_series3_data.csv  
  inflating: train/subj10_series3_events.csv  
  inflating: train/subj10_series4_data.csv  
  inflating: train/subj10_series4_events.csv  
  inflating: train/subj10_series5_data.csv  
  inflating: train/subj10_series5_events.csv  
  inflating: train/subj10_series6_data.csv  
  inflating: train/subj10_series6_events.csv  
  inflating: train/subj10_series7_data.csv  
  inflating: train/subj10_series7_events.csv  
  inflating: train/subj10_series8_data.csv  
  inflating: train/subj10_series8_events.csv  
  inflating: train/subj11_series1_data.csv  
  inflating: train/subj11_series1_events.csv  
  inflating: train/subj11_series2_data.csv  
  inflating: train/subj11_series2_events.csv  
  inflating: train/subj11_series3_data.c

In [0]:
!unzip test.zip

Archive:  test.zip
   creating: test/
  inflating: test/subj10_series10_data.csv  
  inflating: test/subj10_series9_data.csv  
  inflating: test/subj11_series10_data.csv  
  inflating: test/subj11_series9_data.csv  
  inflating: test/subj12_series10_data.csv  
  inflating: test/subj12_series9_data.csv  
  inflating: test/subj1_series10_data.csv  
  inflating: test/subj1_series9_data.csv  
  inflating: test/subj2_series10_data.csv  
  inflating: test/subj2_series9_data.csv  
  inflating: test/subj3_series10_data.csv  
  inflating: test/subj3_series9_data.csv  
  inflating: test/subj4_series10_data.csv  
  inflating: test/subj4_series9_data.csv  
  inflating: test/subj5_series10_data.csv  
  inflating: test/subj5_series9_data.csv  
  inflating: test/subj6_series10_data.csv  
  inflating: test/subj6_series9_data.csv  
  inflating: test/subj7_series10_data.csv  
  inflating: test/subj7_series9_data.csv  
  inflating: test/subj8_series10_data.csv  
  inflating: test/subj8_series9_data.csv  

In [0]:
!unzip sample_submission.csv.zip

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


#Import Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

#First, train the model on all subjects

In [0]:
import numpy as np
import gc
import pandas as pd

 read dataset according to file path

In [0]:
X_train_files_names = []
Y_train_files_names = []
for subj in range(1, 13):
    for series in range(1, 8):
        X_train_files_names.append('./train/subj'+str(subj)+'_series'+str(series)+'_data.csv')

        Y_train_files_names.append('./train/subj'+str(subj)+'_series'+str(series)+'_events.csv')

In [0]:
X_val_files_names = []
Y_val_files_names = []
for subj in range(1, 13):
    series = 8
    X_val_files_names.append('./train/subj'+str(subj)+'_series'+str(series)+'_data.csv')

    Y_val_files_names.append('./train/subj'+str(subj)+'_series'+str(series)+'_events.csv')

In [0]:
X_train = pd.DataFrame()

for file_path in X_train_files_names:
    x = pd.read_csv(file_path)
    X_train = pd.concat([X_train, x])

X_val = pd.DataFrame()

for file_path in X_val_files_names:
    x = pd.read_csv(file_path)
    X_val = pd.concat([X_val, x])

check memory status

In [0]:
!free

Data Preprocessing

In [0]:
X_train.drop('id', axis=1, inplace=True)
X_val.drop('id', axis=1, inplace=True)

def preprocess(col):
    
    mean = col.mean()
    std = col.std()
    col -= mean
    col /=std
    return col

X_train = X_train.apply(preprocess)
X_val = X_val.apply(preprocess)
gc.collect()

In [0]:
X_train = X_train.values
X_val = X_val.values
gc.collect()

read labels

In [0]:
Y_train = pd.DataFrame()
for file_path in Y_train_files_names:
    y = pd.read_csv(file_path)
    Y_train_all = pd.concat([Y_train, y])

In [0]:
Y_val = pd.DataFrame()
for file_path in Y_val_files_names:
    y = pd.read_csv(file_path)
    Y_val = pd.concat([Y_val, y])

In [0]:
Y_train.drop('id', axis=1, inplace=True)
Y_val.drop('id', axis=1, inplace=True)
Y_train = Y_train.values
Y_val = Y_val.values
gc.collect()

In [0]:
print(X_train_std.shape[0], Y_train_all.shape[0], X_val.shape[0], Y_val.shape[0])

Prepare the train data generator

In [0]:
#训练数据集的generator
time_interval = 1024#原数据是500Hz, 取1000个点就是2s的数据
subsamples = 4#每间隔100采样点再踩一次样,Tx=30

def generator(batch_size):
    while 1:
        x = np.zeros((batch_size, time_interval//subsamples, 32))
        y = []
        index = np.random.randint(0, len(X_train)-time_interval-16*batch_size)
        indexes = range(index, index + 16*batch_size, 16)
        for i , index in enumerate(indexes):
            
            x[i] = X_train[index : index+time_interval : subsamples]
            y.append(Y_train[index+time_interval])
        #x = x.reshape(x.shape[0], x.shape[1], x.shape[2],1)    
        y = np.asarray(y)
        yield x, y

In [0]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import AveragePooling1D,LeakyReLU, Lambda, Dense, LSTM, Dropout, CuDNNLSTM, BatchNormalization, Conv2D, Flatten, MaxPooling2D
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import  Dense, LSTM, Dropout, CuDNNLSTM, Conv1D, BatchNormalization, Conv2D, Flatten, MaxPooling2D, MaxPooling1D

savemodel = ModelCheckpoint(filepath='Conv1D.h5')

validation generator

In [0]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

#validation generator
def val_generator(batch_size):
    while 1:
        x = np.zeros((batch_size, time_interval//subsamples, 32))
        y = []
        index = np.random.randint(0, len(X_val)-time_interval-16*batch_size)
        indexes = range(index, index + 16*batch_size, 16)
        for i , index in enumerate(indexes):
            
            x[i] = X_val[index : index+time_interval : subsamples]
            y.append(Y_val[index+time_interval])
        #x = x.reshape(x.shape[0], x.shape[1], x.shape[2],1)    
        y = np.asarray(y)
        yield x, y

#Build the model

First model： 非常简单的一个模型，直接用1D来处理数据，但是最后结果还不错，针对每一个subj进行训练过后，平均AUC 得分能有0.9之上

In [0]:
savemodel = ModelCheckpoint(filepath='Conv1D.h5')

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, strides=1, padding='valid', activation = 'relu', input_shape=[time_interval//subsamples, 32]))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2, stride=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(124, activation='relu'))
model.add(Dropout(0.5))
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid'))

adam = Adam(lr = 0.001)

model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ["accuracy"])

model.summary()

Second model：排名第三用的一种模型，直接输入所有的原数据， window_size = 4096, sub_sample = 1, 让模型自己学习如何采样

In [0]:
savemodel = ModelCheckpoint(filepath='new_model_Conv1D.h5')

model = Sequential()
model.add(Conv1D(filters=6, kernel_size=5, strides=1, padding='same', input_shape=[time_interval//subsamples, 32]))
model.add(Conv1D(filters=6, kernel_size=16, strides=16, padding='valid'))
model.add(Conv1D(filters=32, kernel_size=7, padding='same', strides=1))
model.add(LeakyReLU(alpha=0.1))
model.add(AveragePooling1D(pool_size=3, strides=2))
model.add(Conv1D(filters=32, kernel_size=5, padding='same', strides=1))
model.add(AveragePooling1D(pool_size=12, strides=8))
model.add(Flatten())
model.add(Dropout(0.5))
#model.add(CuDNNLSTM(64))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid'))

adam = Adam(lr = 0.001)

model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ["accuracy"])

model.summary()

Third model，自己想的模型，用LSTM来学习，效果未知

In [0]:
savemodel = ModelCheckpoint(filepath='keras_model/new_model_LSTM.h5')


model = Sequential()
model.add(CuDNNLSTM(128, return_sequences=True, input_shape=[30, 32]))
model.add(CuDNNLSTM(256))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid'))

adam = Adam(lr = 0.001)
#parallel_model = multi_gpu_model(model, gpus=8)

model.compile(loss="binary_crossentropy", optimizer=adam, metrics=['accuracy'])

model.summary()

Forth model

In [0]:
savemodel = ModelCheckpoint(filepath='keras_model/new_model_Conv2D.h5')

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = "same", activation = "relu", input_shape = (time_interval//subsamples, 32, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(32, activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(6, activation = "sigmoid"))
adam = Adam(lr = 0.001)

model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ["accuracy"])

Fith model:当作时空热力图来进行处理，使用2D卷积

In [0]:
model = Sequential()
#model.add(CuDNNLSTM(128, input_shape = (time_steps//subsample, 32)))
model.add(Conv2D(filters = 64, kernel_size = (7,7), padding = "same", activation = "relu", input_shape = (time_interval//subsamples, 32, 1)))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = "same", activation = "relu"))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Flatten())
#model.add(Dropout(0.2))
model.add(Dense(32, activation = "relu"))
model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(Dense(6, activation = "sigmoid"))

adam = Adam(lr = 0.001)

model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ["accuracy"])
model.summary()

In [0]:
batch_size = 2000
steps_per_epcoh = X_train_std.shape[0]//batch_size

model.fit_generator(generator(batch_size), steps_per_epoch = steps_per_epcoh ,epochs=1, callbacks=[savemodel], validation_data=val_generator(64), validation_steps=20, verbose=1)

validation AUC score

In [0]:
#validation
from sklearn.metrics import roc_curve, auc, roc_auc_score

scores = []
num_batch = 10
batch_size = 256

for i in range(num_batch):
    x_val = np.zeros((batch_size, time_interval//subsamples, 32))
    y_val = []
    for i in range(batch_size):
        index = np.random.randint(0, X_val.shape[0]-time_interval)
        x_val[i] = X_val[index : index+time_interval : subsamples]
        y_val.append(Y_val[index+time_interval])
    y_val = np.asarray(y_val)
    #x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], x_val.shape[2],1)   
    y_out = model.predict(x_val)
    score = roc_auc_score(y_val, y_out)
    scores.append(score)
scores = np.asarray(scores)
print(np.mean(scores))

#Start to train the model on individual subject

load data according to subj

In [0]:
X_train_files_names = []
Y_train_files_names = []
for subj in range(1, 13):
    x_subjs = []
    y_subjs = []
    for series in range(1, 8):
        x_subjs.append('./train/subj'+str(subj)+'_series'+str(series)+'_data.csv')

        y_subjs.append('./train/subj'+str(subj)+'_series'+str(series)+'_events.csv')
    X_train_files_names.append(x_subjs)
    Y_train_files_names.append(y_subjs)

In [0]:
X_val_files_names = []
Y_val_files_names = []
for subj in range(1, 13):
    x_subjs = []
    y_subjs = []
    series = 8
    x_subjs .append('./train/subj'+str(subj)+'_series'+str(series)+'_data.csv')

    y_subjs.append('./train/subj'+str(subj)+'_series'+str(series)+'_events.csv')
    X_val_files_names.append(x_subjs)
    Y_val_files_names.append(y_subjs)

In [0]:
import os
os.chdir("/content") 

In [0]:
def preprocess(col):
    
    mean = col.mean()
    std = col.std()
    col -= mean
    col /=std
    return col

For each subj, prepare the train.csv and val.csv

In [0]:
for i in range(12):
    
    X_train = pd.DataFrame()
    
    for file_path in X_train_files_names[i]:
        x = pd.read_csv(file_path)
        X_train = pd.concat([X_train, x])
    
    X_train.drop('id', axis=1, inplace=True)
    print('load X_train'+str(i)+'success')
    
    X_val = pd.DataFrame()
    
    for file_path in X_val_files_names[i]:
        x = pd.read_csv(file_path)
        X_val = pd.concat([X_val, x])
    
    X_val.drop('id', axis=1, inplace=True)
    
    print('load X_val'+str(i)+'success')
    
    X_train = X_train.apply(preprocess)
    X_val = X_val.apply(preprocess)
    gc.collect()
    
    print('process_X_success')
    
    Y_train_all = pd.DataFrame()
    for file_path in Y_train_files_names[i]:
        y = pd.read_csv(file_path)
        Y_train_all = pd.concat([Y_train_all, y])

    Y_train_all.drop('id', axis=1, inplace=True)
    
    print('load Y_train'+str(i)+'success')
   
    Y_val = pd.DataFrame()
    for file_path in Y_val_files_names[i]:
        y = pd.read_csv(file_path)
        Y_val = pd.concat([Y_val, y])
    
    print('load Y_val'+str(i)+'success')
    
    Y_val.drop('id', axis=1, inplace=True)
    
    os.chdir("drive/My Drive/GAL_kaggle") 
    X_train.to_csv('subj'+str(i+1)+'_X_train.csv', index=False)
    print('convert X_train'+str(i)+'success')
    X_val.to_csv('subj'+str(i+1)+'_X_val.csv', index=False)
    print('convert X_val'+str(i)+'success')
    Y_train_all.to_csv('subj'+str(i+1)+'_Y_train.csv', index=False)
    print('convert Y_train'+str(i)+'success')
    Y_val.to_csv('subj'+str(i+1)+'_Y_val.csv', index=False)
    print('convert Y_val'+str(i)+'success')
    os.chdir("/content") 

load X_train0success
load X_val0success
process_X_success
load Y_train0success
load Y_val0success
convert X_train0success
convert X_val0success
convert Y_train0success
convert Y_val0success
load X_train1success
load X_val1success
process_X_success
load Y_train1success
load Y_val1success
convert X_train1success
convert X_val1success
convert Y_train1success
convert Y_val1success
load X_train2success
load X_val2success
process_X_success
load Y_train2success
load Y_val2success
convert X_train2success
convert X_val2success
convert Y_train2success
convert Y_val2success
load X_train3success
load X_val3success
process_X_success
load Y_train3success
load Y_val3success
convert X_train3success
convert X_val3success
convert Y_train3success
convert Y_val3success
load X_train4success
load X_val4success
process_X_success
load Y_train4success
load Y_val4success
convert X_train4success
convert X_val4success
convert Y_train4success
convert Y_val4success
load X_train5success
load X_val5success
process_X_

In [0]:
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


In [0]:
#训练数据集的generator
time_interval = 1024#原数据是500Hz, 取1000个点就是2s的数据
subsamples = 4#每间隔100采样点再踩一次样,Tx=30

def generator(X, Y, batch_size):
    while 1:
        x = np.zeros((batch_size, time_interval//subsamples, 32))
        y = []
        index = np.random.randint(0, len(X)-time_interval-16*batch_size)
        indexes = range(index, index + 16*batch_size, 16)
        for i , index in enumerate(indexes):

            x[i] = X[index : index+time_interval : subsamples]
            y.append(Y[index+time_interval])
        #x = x.reshape(x.shape[0], x.shape[1], x.shape[2],1)    
        y = np.asarray(y)
        yield x, y 

In [0]:
#validation
from sklearn.metrics import roc_curve, auc, roc_auc_score
def val_score(X_val, Y_val, model):
    scores = []
    num_batch = 10
    batch_size = 256

    for i in range(num_batch):
        x_val = np.zeros((batch_size, time_interval//subsamples, 32))
        y_val = []
        for i in range(batch_size):
            index = np.random.randint(0, X_val.shape[0]-time_interval)
            x_val[i] = X_val[index : index+time_interval : subsamples]
            y_val.append(Y_val[index+time_interval])
        y_val = np.asarray(y_val)
        #x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], x_val.shape[2],1)   
        y_out = model.predict(x_val)
        score = roc_auc_score(y_val, y_out)
        scores.append(score)
    scores = np.asarray(scores)

    return np.mean(scores)

Transfer learning, train the model on each subject

In [0]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import pandas as pd
import os
import numpy as np
#os.chdir("drive/My Drive/GAL_kaggle")

for i in range(12):
    X_train = pd.read_csv('subj'+str(i+1)+'_X_train.csv').values
    X_val = pd.read_csv('subj'+str(i+1)+'_X_val.csv').values
    Y_train = pd.read_csv('subj'+str(i+1)+'_Y_train.csv').values
    Y_val = pd.read_csv('subj'+str(i+1)+'_Y_val.csv').values
    
    
    
    print('load_subj'+str(i+1)+' data complete!')
    model = load_model('new_model_Conv1D.h5')
    
    savemodel = ModelCheckpoint(filepath='Conv1D_for_subj'+str(i+1)+'.h5')
                      
    batch_size = 2000
    steps_per_epcoh = X_train.shape[0]//batch_size
    model.fit_generator(generator(X_train, Y_train, batch_size), steps_per_epoch = steps_per_epcoh ,epochs=2, callbacks=[savemodel], verbose=1)
    print('train_model'+str(i+1)+'complete!')
                          
    score = val_score(X_val, Y_val, model)
    print('subj'+str(i+1)+'model AUC scores: '+ str(score))

load_subj1 data complete!
Epoch 1/2
652/652 [==============================] - 92s 141ms/step - loss: 0.0569 - acc: 0.9773
Epoch 2/2
652/652 [==============================] - 87s 133ms/step - loss: 0.0470 - acc: 0.9799
train_model1complete!
subj1model AUC scores: 0.9780302496252625
load_subj2 data complete!
Epoch 1/2
780/780 [==============================] - 107s 137ms/step - loss: 0.0703 - acc: 0.9785
Epoch 2/2
780/780 [==============================] - 107s 137ms/step - loss: 0.0593 - acc: 0.9798
train_model2complete!
subj2model AUC scores: 0.8534114629718786
load_subj3 data complete!
Epoch 1/2
610/610 [==============================] - 85s 140ms/step - loss: 0.0807 - acc: 0.9728
Epoch 2/2
610/610 [==============================] - 85s 139ms/step - loss: 0.0680 - acc: 0.9739
train_model3complete!
subj3model AUC scores: 0.8972959678508733
load_subj4 data complete!
Epoch 1/2
640/640 [==============================] - 89s 139ms/step - loss: 0.0631 - acc: 0.9755
Epoch 2/2
640/640 [====

Prepare the test data

In [0]:
import pandas as pd
import numpy as np
#pred
X_test_files_names = []
for subj in range(1, 13):
    x_subjs = []
    y_subjs = []
    for series in range(9, 11):
        x_subjs.append('./test/subj'+str(subj)+'_series'+str(series)+'_data.csv')

        y_subjs.append('./test/subj'+str(subj)+'_series'+str(series)+'_events.csv')
    X_test_files_names.append(x_subjs)

In [0]:
#pred
X_test = []

for i in range(12):
    x_test = pd.DataFrame()
    x = pd.read_csv(X_test_files_names[i][0])
    x_test = pd.concat([x_test, x])
    x = pd.read_csv(X_test_files_names[i][1])
    x_test = pd.concat([x_test, x])
    x_test.drop('id', axis=1, inplace=True)
    x_test = x_test.apply(preprocess)
    x_test = x_test.values
    X_test.append(x_test)

In [0]:
os.chdir("drive/My Drive/GAL_kaggle")

In [0]:
#pred
from keras.models import load_model
time_interval = 1024#原数据是500Hz, 取1000个点就是2s的数据
subsamples = 4#每间隔100采样点再踩一次样,Tx=30


Y_pred = np.empty([0, 6])
for subj in range(12):
    y_pred = np.zeros((X_test[subj].shape[0], 6))
    pred_batch_size = 128
    batch_index = X_test[subj].shape[0]//pred_batch_size
    
    current_index = 0
    model = load_model('Conv1D_for_subj'+ str(subj+1) +'.h5')
    for index in range(batch_index):
        x_test = np.zeros((pred_batch_size, time_interval//subsamples, 32))
        for i in range(pred_batch_size):
            if current_index-time_interval < 0:
                padding = np.zeros((time_interval-current_index, 32))
                x = np.append(padding, X_test[subj][:current_index], axis=0)
                x_test[i] = x[::subsamples]
            else:    
                x_test[i] = X_test[subj][current_index-time_interval : current_index: subsamples]
            
            current_index += 1
        
        y_pred[index*pred_batch_size : (index+1)*pred_batch_size] = model.predict(x_test)
        
        
        if index % 50 == 0:
            print(index)
    
    the_rest_batch_size = X_test[subj].shape[0]-batch_index*pred_batch_size
    x_test = np.zeros((the_rest_batch_size, time_interval//subsamples, 32))
    for i in range(the_rest_batch_size):
        x_test[i] = X_test[subj][current_index-time_interval : current_index : subsamples]
        current_index += 1
    
    y_pred[batch_index*pred_batch_size : ] = model.predict(x_test)
    
    Y_pred = np.append(Y_pred, y_pred, axis=0)
    print(X_test[subj].shape[0], current_index, current_index==X_test[subj].shape[0])

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
233081 233081 True
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
298085 298085 True
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
225339 225339 True
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
244794 244794 True
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
261060 261060 True
0
50
100
150
200
250

In [0]:
os.chdir("/content")
!unzip sample_submission.csv.zip

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


In [0]:
submission = pd.read_csv('sample_submission.csv')

In [0]:
submission.shape[0] 

3144171

In [0]:
Y_pred.shape[0]

3144171

In [0]:
submission.loc[:, ['HandStart', 'FirstDigitTouch', 'BothStartLoadPhase', 'LiftOff', 'Replace', 'BothReleased']]=Y_pred


In [0]:
submission.to_csv('conv1D_for_subj.csv', index=False)

In [0]:
!kaggle competitions submit -c grasp-and-lift-eeg-detection -f conv1D_for_subj.csv -m "Message"

100% 308M/308M [00:08<00:00, 38.4MB/s]
Successfully submitted to Grasp-and-Lift EEG Detection